## Python for REU 2019

_Burt Rosenberg, 7 May 2019_


### Various thing about lists



The list is one of the most versatile datastructures. It models a sequence of items, which can be a mix of types.

Items can be accessed through the index of the item on the list. The first item is at index 0. For a list L access the i-th item with square brackets: L[i]. The last item on the list is at index -1, and the previous to last, at -2, and so on.

**Slices**

Python supports the selection of multiple elements by index using *slice* notation. The full notation is [a:b:c] fo for a slace beginning at a, ending before b, advancing to include only every c-th element.

Slice notation can appear on the right hand side of an assignment, to retrieve a slice of elements, or on the left hand side of an assignment, to receive a slice of elements.

Leaving a location blank refers to the default:

* the default for the start is the first item in the list
* the default for the end is the list item in the list
* the default for the skip is to take all items in the start-end range


Here are some fun tricks with slices:

In [1]:
def fun_slice():
    """
    here are some fun slice tricks.
    """
    a = [i for i in range(10)]
    print(a)
    print(a[0],a[-1])
    print(a[::2])
    print(a[::-2])

    b = [-i for i in range(10)]
    print(b)
    b[::2] = a[::2]
    print(b)
    b[::2] = a[::-2]
    print(b)
    
    a[len(a)//2:] = a[len(a)//2::-1]
    print(a)
    
fun_slice()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0 9
[0, 2, 4, 6, 8]
[9, 7, 5, 3, 1]
[0, -1, -2, -3, -4, -5, -6, -7, -8, -9]
[0, -1, 2, -3, 4, -5, 6, -7, 8, -9]
[9, -1, 7, -3, 5, -5, 3, -7, 1, -9]
[0, 1, 2, 3, 4, 5, 4, 3, 2, 1, 0]



__Shallow Copies__

There is an important difference between a slice of a list, and the list. A slice extracts values from the list and copies them over. 

We start with a simple example:

In [2]:
a = [i for i in range(10)]
b1 = a
b2 = a[:]
a[::2] = a[::-2]
print(a)
print(b1)
print(b2)

[9, 1, 7, 3, 5, 5, 3, 7, 1, 9]
[9, 1, 7, 3, 5, 5, 3, 7, 1, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


By using a slice to transfer values, the b2 array is not modified by changes in a, although b1 is. This is called a shallow copy, however, because changes in a can still change b2. If what is copied is a referene, rather than a value, changes in what the reference refers to is visible in the copy. For instance:

In [3]:
a = [[i] for i in range(10)]
b2 = a[:]
for i in range(len(a)): a[i][0] = -a[i][0]
print(a)
print(b2)

[[0], [-1], [-2], [-3], [-4], [-5], [-6], [-7], [-8], [-9]]
[[0], [-1], [-2], [-3], [-4], [-5], [-6], [-7], [-8], [-9]]


__Deep Copies__

A *deep copy* would descend recursively into each list and use the slice notation (among other methods) to copy values rather than references. The more complicated RememberMe class summarizes this.

In [4]:
class RememberMe:
    
    def __init__(self):
        pass
           
    def just_remember(self,o):
        self.that_object = o

    def copy_shallow(self,o):
        self.list_shallow = o[:]
 
    def copy_deep_aux(self,o):
        # if o is a list
        if isinstance(o,[].__class__):
            # shallow copy the backbone
            a = o[:]
            for i, b in enumerate(a):
                # and recurse on each element
                a[i] = self.copy_deep_aux(b)
            return a
        else:
            return o
            
    def copy_deep(self,o):
        self.list_deep = self.copy_deep_aux(o)      
        
    def tell_all(self):
        return (self.that_object,self.list_shallow,self.list_deep)

test_list = [1,2,3,[1,2,3],[[1,2,3],[1,2,3],[1,2,3]]]

r_m = RememberMe()
r_m.just_remember(test_list)
r_m.copy_shallow(test_list)
r_m.copy_deep(test_list)

test_list[0] = 'a'
test_list[3][0] = 'b'
test_list[4][0][0] = 'c'
(a,b,c) = r_m.tell_all()
print("saved the reference", a)
print("made a shallow copy", b)
print("made a deep copy", c)


saved the reference ['a', 2, 3, ['b', 2, 3], [['c', 2, 3], [1, 2, 3], [1, 2, 3]]]
made a shallow copy [1, 2, 3, ['b', 2, 3], [['c', 2, 3], [1, 2, 3], [1, 2, 3]]]
made a deep copy [1, 2, 3, [1, 2, 3], [[1, 2, 3], [1, 2, 3], [1, 2, 3]]]


**Using a list as an iterator**

A list is iterable, so it can be used in a for statment exactly in place of the range() function.

It isn't exactly as if the list iterates, but that it wrap itself inside an iterator. If need be, you can explicitly implement to for loop by going from a list to an iterator with the iter function, then apply the next function to the iterator, and make sure that you catch the StopIteration exception.

To go from an iterator to a list, use the list function.

In [38]:
a_l = [i for i in range(10)]
for e in a_l:
    print(e)

a_l_itr = iter(a_l)
try:
    while True:
        print(next(a_l_itr))
except StopIteration:
    pass

a_l_itr = iter(a_l)
print(list(a_l_itr))

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]



### A few list functions, briefly

In [41]:
l = [3,6,9,1,2,7]

# min
print("min",min(l),"max",max(l))

#append
l.append(13)
print(l)

try:
    print([1,2,3].index(2))
    [1,2,3].index(4)
except ValueError:
    print("wasn't there")
    
try:
    l = [1,2,2,3]
    l.remove(2)
    print(l)
    [1,2,3].remove(4)
except ValueError:
    print("wasn't there")


min 1 max 9
[3, 6, 9, 1, 2, 7, 13]
1
wasn't there
[1, 2, 3]
wasn't there


### Exercise: Selection Sort

Please write a selection sort, which works inplace &mdash; the numbers are rearranged in the same array, not copied to a new array. 

The helpful loop invariant is that the loop begins with i indexing the next location to be finalized, beginning with i = 0; and the value to place in location i is found among locations j>=i. If the value to put in location i comes from location j, swap values between locations i and j.


In [5]:

# fix my broken code

def selection_sort(a):
    """
    selection sort the list a
    """
    for i in range(len(a)-1):
        # i is where to place the smallest number amount [i:]
        pass
    return a

def test_selection_sort():
    test = [(13*i)%97 for i in range(84)]
    ans = sorted(test[:])
    selection_sort(test)
    if test == ans:
        print("correct!")
    else:
        print("broken!")   

test_selection_sort()

correct!


### Exercise: Insertion Sort

Please write an insertion sort. The sort by its nature works in-place &mdash; the values are moved around the array, not copied off to a new array.

The Loop Invariant is that at the top of the loop, location i is considered for swapping with location i+1, to bring the smaller value to location i. Additionally j is set initially to i and the inner loop invariant 

In [6]:
#fix my broken code

def insertion_sort(c):
    if len(c)== 0 : return c
    
    for i in range(len(c)-1):
        for j in range(i,-1,-1):
            pass
    return c

def test_insertion_sort():
    test = [(13*i)%97 for i in range(84)]
    ans = sorted(test[:])
    insertion_sort(test)
    if test == ans:
        print("correct!")
    else:
        print("broken!")   

test_insertion_sort()

correct!
